In [14]:
import os
import requests
import jwt
import time

def generate_token(apikey:str,exp_seconds:int):
    try:
        id,secret=apikey.split(".")
    except Exception as e:
        raise Exception("Invalid apikey",e)
    
    payload={
         "api_key":id,
         "exp":int(round(time.time()*1000))+exp_seconds*1000,
        "timestamp":int(round(time.time()*1000))
    }

    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg":"HS256","sign_type":"SIGN"}
    )


api_key=os.environ["ZHIPUAI_API_KEY"]
token=generate_token(api_key,60)
url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
headers={
     "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}
data = {
    "model": "glm-4",
    "messages": [
        {
            "role": "system",
            "content": "your are a helpful assistant"
        },
        {
            "role": "user",
            "content": "can you tell me a joke?"
        }
    ],
    "max_tokens": 8192,
    "temperature": 0.8,
    "stream": False
}

response=requests.post(url,headers=headers,json=data)
ans=response.json()

In [12]:
ans

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nI hope that brings a smile to your face! If you want to hear more jokes, feel free to ask.",
    'role': 'assistant'}}],
 'created': 1710818674,
 'id': '8489586804394253002',
 'model': 'glm-4',
 'request_id': '8489586804394253002',
 'usage': {'completion_tokens': 47, 'prompt_tokens': 19, 'total_tokens': 66}}

In [3]:
print(ans["choices"][0]["message"]["content"])
print(ans["id"])

Here's a light-hearted joke for you:

Why don't scientists trust atoms?

Because they make up everything!

I hope that brings a smile to your face! If you want to hear more jokes, feel free to ask.
8489586804394253002


In [17]:
data["stream"]=False
api_key=os.environ["ZHIPUAI_API_KEY"]
token=generate_token(api_key,60)
url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
headers={
     "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}
print(data)
response=requests.post(url,headers=headers,json=data)
ans=response.json()
ans



{'model': 'glm-4', 'messages': [{'role': 'system', 'content': 'your are a helpful assistant'}, {'role': 'user', 'content': 'can you tell me a joke?'}], 'max_tokens': 8192, 'temperature': 0.8, 'stream': False}


{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': "Sure, here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!",
    'role': 'assistant'}}],
 'created': 1710819280,
 'id': '8489581822231979837',
 'model': 'glm-4',
 'request_id': '8489581822231979837',
 'usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45}}

In [ ]:
for chunk in response.iter_lines():
    if chunk:
        chunk_str=chunk.decode("utf-8")
        json_start_pos=chunk_str.find('{"id"}')
        if json_start_pos!=-1:
              json_str=chunk_str[json_start_pos:]
              json_data=json.loads(json_str)
              for choice in json_data.get("choices",[]):
                delta=choice.get('delta',{})
                content=delta.get('content',{})
                print(content,end='|')